In [48]:
api_key = "5641715277706f70363756586c5777"

In [49]:
import requests
import csv

# API 기본 URL
base_url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/bikeList/"
start_index = 1
end_index = 1000
all_stations = []

# 데이터 수집
while True:
    # API 요청 URL 생성
    url = f"{base_url}{start_index}/{end_index}/"
    response = requests.get(url)

    try:
        # JSON 데이터 파싱
        data = response.json()

        # 데이터가 없거나 오류가 발생하면 반복 종료
        if 'rentBikeStatus' not in data or 'row' not in data['rentBikeStatus']:
            break

        # 수집된 데이터를 리스트에 추가
        stations = data['rentBikeStatus']['row']
        all_stations.extend(stations)

        # 다음 요청을 위해 인덱스 갱신
        start_index += 1000
        end_index += 1000
    except Exception as e:
        # 오류가 발생하면 반복 종료
        print(f"데이터 수집 중 오류 발생: {e}")
        break

# CSV 파일로 저장
file_path = '/content/bike_stations_from_api.csv'
with open(file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    # 헤더 작성
    writer.writerow(['stationId', 'stationName', 'latitude', 'longitude'])

    # 데이터 작성
    for station in all_stations:
        writer.writerow([station['stationId'], station['stationName'], station['stationLatitude'], station['stationLongitude']])

print(f"CSV 파일이 성공적으로 저장되었습니다: {file_path}")

# Colab에서 다운로드 링크 생성
from google.colab import files
files.download(file_path)


CSV 파일이 성공적으로 저장되었습니다: /content/bike_stations_from_api.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [50]:
import pandas as pd
import math


# 하버사인 공식을 사용하여 두 위도/경도 점 사이의 거리를 계산합니다.
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # 지구의 반경 (km)
    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat / 2) ** 2 + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    return R * c  # 거리 (km)

# 쿼드 트리 노드 클래스 정의
class QuadTreeNode:
    def __init__(self, x_min, x_max, y_min, y_max, capacity):
        self.boundary = (x_min, x_max, y_min, y_max)
        self.capacity = capacity
        self.points = []
        self.divided = False
        self.northwest = None
        self.northeast = None
        self.southwest = None
        self.southeast = None

    # 노드 분할
    def subdivide(self):
        x_min, x_max, y_min, y_max = self.boundary
        x_mid = (x_min + x_max) / 2
        y_mid = (y_min + y_max) / 2

        self.northwest = QuadTreeNode(x_min, x_mid, y_mid, y_max, self.capacity)
        self.northeast = QuadTreeNode(x_mid, x_max, y_mid, y_max, self.capacity)
        self.southwest = QuadTreeNode(x_min, x_mid, y_min, y_mid, self.capacity)
        self.southeast = QuadTreeNode(x_mid, x_max, y_min, y_mid, self.capacity)
        self.divided = True

    # 포인트 삽입
    def insert(self, point):
        x, y = point
        x_min, x_max, y_min, y_max = self.boundary
        if not (x_min <= x <= x_max and y_min <= y <= y_max):
            return False

        if len(self.points) < self.capacity:
            self.points.append(point)
            return True
        else:
            if not self.divided:
                self.subdivide()

            if self.northwest.insert(point): return True
            if self.northeast.insert(point): return True
            if self.southwest.insert(point): return True
            if self.southeast.insert(point): return True

    # 특정 거리 내의 포인트 검색
    def query_range(self, lat, lon, radius, found_points):
        x_min, x_max, y_min, y_max = self.boundary

        # 사각형 경계와 탐색 영역의 교차 여부 확인
        if not (x_min <= lon + radius and x_max >= lon - radius and y_min <= lat + radius and y_max >= lat - radius):
            return

        # 현재 노드의 모든 포인트 검사
        for px, py in self.points:
            if haversine(lat, lon, py, px) <= radius:
                found_points.append((py, px))

        # 하위 노드 탐색
        if self.divided:
            self.northwest.query_range(lat, lon, radius, found_points)
            self.northeast.query_range(lat, lon, radius, found_points)
            self.southwest.query_range(lat, lon, radius, found_points)
            self.southeast.query_range(lat, lon, radius, found_points)

# CSV 파일을 읽어 따릉이 보관소 위치 정보를 가져옵니다.
file_path = '/content/bike_stations_from_api.csv'
df = pd.read_csv(file_path, encoding='utf-8')

# 쿼드 트리 생성 및 데이터 삽입
quad_tree = QuadTreeNode(x_min=-180, x_max=180, y_min=-90, y_max=90, capacity=4)
for index, row in df.iterrows():
    latitude = row['latitude']
    longitude = row['longitude']
    quad_tree.insert((longitude, latitude))

# 현재 위치 및 검색 반경 (km)
current_location = (37.50126, 127.03948)  # 서울의 예제 위치
search_radius = 5.0  # 2km 이내

# 특정 거리 이내의 보관소 검색
found_stations = []
quad_tree.query_range(current_location[0], current_location[1], search_radius, found_stations)

# 검색된 보관소 출력
print("검색된 보관소:", found_stations)


검색된 보관소: [(37.54122162, 127.04380035), (37.54458237, 127.04460907), (37.54205322, 127.020401), (37.54136658, 127.01776123), (37.54404068, 127.02320099), (37.54281616, 127.0446701), (37.54590988, 127.04460144), (37.54281616, 127.04208374), (37.54195786, 127.04277039), (37.54255676, 127.04423523), (37.53016663, 127.00743866), (37.53514099, 127.00600433), (37.53591919, 127.00541687), (37.52930069, 127.03559875), (37.53075027, 127.04160309), (37.53730774, 127.07035065), (37.53186035, 127.06719208), (37.54317856, 127.0490799), (37.54127502, 127.04849243), (37.53680801, 127.05548859), (37.53268814, 127.06749725), (37.53701401, 127.06109619), (37.5369606, 127.06273651), (37.5297699, 127.07485962), (37.5352211, 127.06839752), (37.53520966, 127.06900024), (37.53507233, 127.07611084), (37.53298569, 127.07579041), (37.53181076, 127.08074188), (37.520504, 126.98973846), (37.52033615, 126.9942627), (37.5240097, 127.00144958), (37.51990891, 126.98923492), (37.5251503, 126.99208832), (37.52529526, 12

In [51]:
import folium

# 현재 위치 및 검색 반경 (km)
current_location = (37.50095209017989, 127.03879023188505)  # 역삼 멀캠
search_radius = 20.5  # 5km 이내

# 특정 거리 이내의 보관소 검색
found_stations = []
quad_tree.query_range(current_location[0], current_location[1], search_radius, found_stations)

# folium을 사용해 지도 생성 및 마커 추가
map_object = folium.Map(location=current_location, zoom_start=15)

# 현재 위치에 마커 추가
folium.Marker(current_location, popup='현재 위치', icon=folium.Icon(color='red')).add_to(map_object)

# 검색된 보관소에 마커 추가
for lat, lon in found_stations:
    folium.Marker([lat, lon], popup='따릉이 보관소').add_to(map_object)

# 지도를 출력합니다.
map_object